# Political Boundaries and Simulations

## Zoë Farmer

# What are we talking about?

In this presentation I'll discuss two numerical approaches to solving political boundaries while striving to avoid gerrymandering.

Slides, code, and images are available here if you want to follow along.

http://bit.ly/2xvaHnX

or

http://dataleek.io/presentations/politicalboundaries

# Who am I?

* My name is Zoë Farmer
* I'm a recent CU graduate with a BS in Applied Math and a CS Minor
* I'm a co-coordinator of the Boulder Python Meetup
* I'm a big fan of open source software
* I'm looking for work (pls hire me)
* http://www.dataleek.io
* http://www.twitter.com/thedataleek
* [git[hub|lab].com/thedataleek](http://github.com/thedataleek)

# Our Algorithms

* [Simulated Annealing](https://en.wikipedia.org/wiki/Simulated_annealing)
    * Minor permutations in a given solution until we find a solution that's slightly better, and repeat.
* [Genetic Algorithm](https://en.wikipedia.org/wiki/Genetic_algorithm)
    * Create a ton of random solutions, have them "combine" and take the best of their children.

# Simulated Annealing

1. Generate a random solution
2. Generate a "neighboring solution" to our generated solution
3. Keep whichever is better, or (with decaying probability) take the new one regardless
4. Go back to 2

In [6]:
def simulated_annealing(system):
    solution, value = generate_solution(system)  # Initial solution
    time = 1
    Tmin = 1e-9
    alpha = 0.99
    k = 1  # larger k => higher probability of randomly accepting a solution
    i = 0
    while T > Tmin:
        solution_2, value_2 = generate_solution_neighbor(system, solution, value)
        value_delta = value_2 - value
        if value_delta < 0:
            solution = solution_2
            value = value_2
        elif random.random() < math.exp(-value_delta / (k * T)):
            solution = solution_2
            value = value_2
        T *= alpha
        i += 1
    print(solution, value, i)

# Genetic Algorithm

1. Randomly generate an initial population of solutions
2. Use our solution population to generate some large number of children (note,
   these children should inherit properties from their parents)
3. Keep the best of our total population
4. Go back to 2

In [9]:
def genetic_algorithm(system):
    solutions = generate_3_solutions(system)
    for i in range(precision):
        new_solutions = []
        for _ in range(10):  # Create 10 children per frame
            s1, s2 = np.random.choice(solutions, size=2) # Randomly combine two parents
            new_solutions.append(s1.combine(s2))
        full_solutions = new_solutions + solutions
        # Keep the top 3 for next generation
        solutions = get_top_3(full_solutions)
    return solutions

# Drawing Political District Boundaries

How can this be applied to political boundaries?

Assumptions:

* 2 parties
* Rectangular areas
* Provided in a specific format

```
D R D R D R R R
D D R D R R R R
D D D R R R R R
D D R R R R D R
R R D D D R R R
R D D D D D R R
R R R D D D D D
D D D D D D R D
```

Which can be plotted for readability.

![png](./img/smallState_initial.png)

# Related Problems to Solve First

## Finding Neighbors of a Point

Our first big problem is how we find neighbors of a single point. For any `(y, x)` pair we can express its neighbors using the following algorithm.

1. Iterate over range(-1, 2) for both x and y
2. For each loop, accept (y + yi, x + xi) if the following conditions hold:
    * y + yi is within the range of the field
    * x + xi is within our domain of the field
    * xi and yi are not both equal to zero

In [14]:
def point_neighbors(y0, x0):
    neighbors = [(y0 + y, x0 + x)
                 for x in range(-1, 2)
                 for y in range(-1, 2)
                 if (0 <= y0 + y < self.height) and
                 (0 <= x0 + x < self.width) and
                 not (x == 0 and y == 0)]
    return neighbors

## Determining if a District is Valid

How do we determine valid solutions?

* Think of these as [single connected components](https://en.wikipedia.org/wiki/Connected_component_(graph_theory))
* We can use [connected component labelling](https://en.wikipedia.org/wiki/Connected-component_labeling). (from wikipedia)

> 1. Start from the first pixel in the image. Set "curlab" (short for "current label") to 1. Go to (2).
> 2. If this pixel is a foreground pixel and it is not already labelled, then give it the label "curlab" and add it as the first element in a queue, then go to (3). If it is a background pixel or it was already labelled, then repeat (2) for the next pixel in the image.
> 3. Pop out an element from the queue, and look at its neighbours (based on any type of connectivity). If a neighbour is a foreground pixel and is not already labelled, give it the "curlab" label and add it to the queue. Repeat (3) until there are no more elements in the queue.
> 4. Go to (2) for the next pixel in the image and increment "curlab" by 1.

In [15]:
class Mask(object):
    # ...
    def get_labels(self):
        curlab = 1
        labels = np.zeros(self.mask.shape)
        q = queue.Queue()
        def unlabelled(i, j):
            return self.mask[i, j] == 1 and labels[i, j] == 0
        for i in range(self.height):
            for j in range(self.width):
                if unlabelled(i, j):
                    labels[i, j] = curlab
                    q.put((i, j))
                    while not q.empty():
                        y0, x0 = q.get()
                        neighbors = [(y0 + y, x0 + x)
                                     for x in range(-1, 2)
                                     for y in range(-1, 2)
                                     if (0 <= y0 + y < self.height) and
                                     (0 <= x0 + x < self.width) and
                                     not (x == 0 and y == 0)]
                        for ii, jj in neighbors:
                            if unlabelled(ii, jj):
                                labels[ii, jj] = curlab
                                q.put((ii, jj))
                    curlab += 1
        return curlab, labels
    # ...

## Finding District Neighbors

* Need to find all neighbors of a given district
* Super similar to connected component labelling

The basic algorithm is as follows.

1. Get a random spot inside the given district
2. Add this spot to a Queue
3. Initialize an empty labelling array (as with connected component labelling)
4. While the queue is not empty, get an new `(y, x)` pair.
5. If the point falls within the district, get all of the point's neighbors, add them to the queue, and go back to (4)
6. If the point does not fall into the district, add it to the list of district neighbors.

In [16]:
class Solution(object):
    # ...
    def get_district_neighbors(self, i):
        y, x = self.get_random_openspot(i)
        q = queue.Queue()
        q.put((y, x))
        edges = []
        labels = np.zeros(self.full_mask.shape)
        labels[y, x] = 1
        while not q.empty():
            y, x = q.get()
            if self.full_mask[y, x] == i:
                for yi, xi in self.get_neighbors(y, x):
                    if labels[yi, xi] == 0:
                        q.put((yi, xi))
                        labels[yi, xi] = 1
            else:
                edges.append((y, x))
        return edges
    # ...

## What is a Fitness Function?

For both these algorithms we talk about their "value", which in this case is determined with a fitness function.

From wikipedia:

> A fitness function is a particular type of objective function that is used to summarise, as a single figure of merit, how close a given design solution is to achieving the set aims.

TL;DR a single number that basically tells us how "good" of a solution we have.

Taking a step back from the code and considering the real world, let's think about what we'd ideally like to emphasize in a political districting system.

* We'd want districts to be homogeneous.
* We want our district ratios to approximately match our population ratios.
* We'd want to avoid [gerrymandering](https://en.wikipedia.org/wiki/Gerrymandering)
<img src="./img/gerrymandering_example.jpg" style="width: 400px; height: 300px;"/>

* We want all districts to be around the same population size.

Translated to our code these priorities become

1. Validity of solution
2. Make sure the ratio of `R` to `D` majority districts matches the ratio of `R` to `D` in the general population.
3. Make sure each district is as homogeneous as possible
4. Reduce the value of the district if its size isn't close to the "ideal size", which is `total_size / num_districts`.
5. We also take into account that in non-homogeneous districts voters that aren't affiliated with the majority party might be swayed by targeted campaigns. To this effect we account each non-affiliated "zone" with a weight of -0.9 instead of -1.
6. Finally, we can also minimize edge length as well as trying to keep each district the same size. This will result in hopefully ideal districts

In [19]:
class Solution(object):
    # ...
    @property
    def value(self):
        value = 0
        if not self.is_valid:  # if we don't have a valid solution, return 0
            return 0
        # num districts tries to match population distribution within 10%
        size, stats = self.system.stats
        for k, v in self.majorities.items():
            if np.abs((float(v) / self.numdistricts) - stats[k]) >= 0.1:
                return 0
        district_size = int(self.width * self.height / self.numdistricts)
        # Sum up values of each district
        for i in range(1, self.numdistricts + 1):
            values = self.system.matrix[self[i].mask.astype(bool)]
            if len(values) == 0:
                return 0
            else:
                # District value is simply abs(num_red - num_blue)
                subvalue = np.abs(len(values[values == 0]) -
                                  len(values[values == 1]))
                size_bonus = 0.25 * np.abs(len(values) - district_size)
                if subvalue < len(values):
                    # For any non-uniform values, add 10% their value to account
                    # for independent voter turnout
                    subvalue += (len(values) - subvalue) * 0.1
                value += subvalue
                value -= size_bonus
                # Minimize neighbors (same as minimizing edge length)
                value += -0.1 * len(self.get_district_neighbors(i))
        return value
    # ...

## Generating Random Solutions

This algorithm is very straightforward.

1. Generate a number of "spawn points" equal to the number of districts.
2. Fill.

The fill algorithm is also straightforward.

1. Set a list of available districts.
2. While there are any non-set points, pick a random district, `i`, from the list of available districts.
3. Get a list of all neighbors of the district, but filter to only 0-valued entries.
4. If no such neighbors exist, remove this district from the list of available districts.
5. Otherwise pick a neighbor at random and set it to `i`.
6. Loop back to (2).

In [20]:
class Solution(object):
    # ...
    def generate_random_solution(self, history=False):
        solution_history = [self.copy()]
        for i in range(1, self.numdistricts + 1):
            y, x = self.get_random_openspot(0)
            self.full_mask[y, x] = i
            if history:
                solution_history.append(self.copy())
        solution_history += self.fill(keep_history=history)
        if history:
            return solution_history
    # ...

![gif](./img/generate_random_solution_smallState.gif)

# Simulated Annealing

Recall:

1. Generate a random solution
2. Generate a solution neighbor
3. If the new solution is better than the old, set the current solution to the new one.
4. Sometimes accept a worse solution

In [21]:
def simulated_annealing(system, numdistricts, precision):
    solution = get_good_start(system, numdistricts)
    k = 0.25  # Larger k => more chance of randomly accepting
    Tvals = np.arange(1, 1e-12, -1.0 / precision)
    print(f'Running Simulated Annealing with k={k:0.03f}, alpha={1.0 / precision:0.05f}')
    for i, T in tqdm(enumerate(Tvals), total=len(Tvals)):
        new_solution = solution.copy()  # copy our current solution
        new_solution.mutate()  # Mutate the copy
        dv = new_solution.value - solution.value  # Look at delta of values
        # If it's better, or random chance, we accept it
        if dv > 0 or random.random() < math.exp(dv / (k * T)):
            solution = new_solution

## How much does choice of `k` impact solution selection?

![png](./img/kvals.png)

The entire process looks like this:

![gif](./img/simulated_annealing_solution_smallState.gif)

Which has the following final solution.

![png](./img/simulated_annealing_solution_smallState.png)

## Mutations

Simulated Annealing relies on "mutating" solutions via the following algorithm.

1. Find all district neighbors
2. Pick a neighboring point at random.
3. If the neighboring point's district has at least size 2, set this neighboring point to our district.
4. Otherwise, pick a different neighboring point.

In [28]:
class Solution(object):
    # ...
    def mutate(self):
        i = np.random.randint(1, self.numdistricts)
        y, x = self.get_random_openspot(i)
        traversed = set()
        q = queue.Queue()
        q.put((y, x))
        while not q.empty():
            y, x = q.get()
            if (y, x) not in traversed:
                traversed.add((y, x))

                if (self.full_mask[y, x] != i and
                        self[self.full_mask[y, x]].size > 1):
                    old_value = self.full_mask[y, x]
                    self.full_mask[y, x] = i
                    if not self.is_valid:  # make sure new mutation is valid
                        # If not, reset and start over
                        self.full_mask[y, x] = old_value
                    else:
                        break

                for ii, jj in self.get_neighbors(y, x):
                    q.put((ii, jj))
    # ...

Which can be visualized as follows.

![png](./img/mutation.png)

# Genetic Algorithms

As simulated annealing relies on `mutate()` to narrow down on a good solution, the genetic algorithm relies on `combine()` to take two solutions and generate a "child" solution.

1. Shuffle our two parents in an array.
2. Shuffle a list of districts.
3. Set a cursor that points to the first parent in the array.
4. Iterate through our districts with variable `i`
5. For the current district, find all points of the parent that our cursor is pointing to.
6. Get all "open" (i.e. set to 0) points for our child solution
7. For every point that matches between these two sets, make a new bitmask.
8. If this bitmask is valid (i.e. one connected component), set all point in this child solution to our current district
9. Otherwise, make the district valid and set the bits in the child solution
10. Flip the cursor

The algorithm behind making a district valid is easy, if we have more than one
connected component in a given district, pick one at random and discard the
other connected components.

In [29]:
class Solution(object):
    # ...
    def combine(self, other_solution):
        new_solution = Solution(self.system, self.numdistricts)
        # Randomly order parents to choose from
        pick_order = [self, other_solution]
        random.shuffle(pick_order)
        # Randomly order districts to choose from
        districts = list(range(1, self.numdistricts + 1))
        random.shuffle(districts)
        cursor = 0  # alternates between parents
        for i in districts:
            parent_locations = pick_order[cursor][i].location
            open_locations = new_solution.get_full_openspots(0)
            district = Mask()
            # We make every child valid
            district.parse_locations(self.height, self.width,
                                     [(y, x) for y, x in parent_locations
                                      if [y, x] in open_locations])
            if not district.is_valid:
                district.make_valid()
            for y, x in district.location:
                new_solution.full_mask[y, x] = i
            cursor ^= 1
        for i in range(1, self.numdistricts + 1):
            y, x = new_solution.get_random_openspot(i)
            if y is None:
                y, x = new_solution.get_random_openspot(0)
                new_solution.full_mask[y, x] = i
        if random.random() < 0.1:
            new_solution.mutate()
        return new_solution
    # ...

Which can be visualized as follows.

![png](./img/combine.png)

# Final Thoughts

* They're both unique approaches that can be applied to incredibly complex problems
* However much of their success hinges on the effectiveness of your fitness function
* Any given "final solution" is somewhat unique, or at the very least improbable to obtain again

# Using the Code

This is straightforward! After installing the required libraries (check [the repository](https://gitlab.com/thedataleek/politicalboundaries)) just run

```bash
$ python3.6 ./politicalboundaries.py $FILE_TO_RUN
```

If you want to dig a little deeper, use the `-h` flag to see what it can do, but
here's a short list as well.

* Use Simulated Annealing on the file
* Use the Genetic Algorithm on the file
* Set the number of districts for either solution type
* Set the precision (number of runs) for either algorithm
* Animate the solution process
* Create gifs of the solution process (otherwise just `.mp4`)
* Generate report (`README.md`) assets.
* Do all of the above in one go.

# Next Steps

I want to do more for this project but I'm limited in the time I have. I do have
a couple of ideas for next steps however.

* Parallelizing - Instead of just running simulations on a single thread, we could theoretically spin up a bunch of different threads and run simulations on them simultaneously, only keeping the best of all trials.
* Real Data - It would be amazing to take the approaches used in this writeup and apply it to real-world political data.

# Questions?